In [1]:
import geopandas as gpd
from shapely.geometry import Polygon
import numpy as np
import pandas as pd

In [145]:
data1 = pd.read_csv("seasonal_crops_Farm_Africa_data_2019.csv")

In [146]:
data1.head()

,row_loc,col_loc,tile,fid,NDVI_0,NDVI_1,NDVI_2,NDVI_3,NDVI_4,NDVI_5,NDVI_6,NDVI_7,NDVI_stand_dev,seasonal_crops
0,4248,2323,0,14318083,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000000,0
1,4250,956,0,14323456,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000000,0
2,2089,1571,0,7041501,0.157895,0.238095,0.315789,0.050000,0.240506,0.327869,0.344828,0.327273,0.102866,1
3,1797,78,0,6055968,0.892857,0.846154,0.825000,0.839080,0.855422,0.885714,0.875000,0.863014,0.023451,0
4,3543,3369,0,11943279,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000000,0


In [147]:
#getting NDVI columns from the whole dataframe
selectlist_1 =[ 'NDVI_0', 'NDVI_1', 'NDVI_2', 'NDVI_3', 'NDVI_4', 'NDVI_5', 'NDVI_6', 'NDVI_7']
datatowrite = data1[selectlist_1]
max_value = datatowrite.to_numpy().max()
min_value = datatowrite.to_numpy().min()

print( max_value)
print( min_value)

1.0
-1.0


In [148]:
# for i in range(11) :
#     df['NDVI_Standardise_{}'.format(i)] = ((df['NDVI_{}'.format(i)]-(-0.3991874))/(0.9409440-(-0.3991874))*1)
#     df['NDVI_Standardise_{}'.format(i)] = df['NDVI_Standardise_{}'.format(i)].fillna(0.000001)

In [149]:
# data1['seasonal_crops_2'] = data1['NDVI_stand_dev'].apply(lambda x :1 if (x>0.05) else 0)

In [150]:
data1.shape

(14325870, 14)

In [151]:
# csv = data1.drop(columns=['row_loc', 'col_loc','tile'])

In [152]:
data1.columns

Index(['row_loc', 'col_loc', 'tile', 'fid', 'NDVI_0', 'NDVI_1', 'NDVI_2',
       'NDVI_3', 'NDVI_4', 'NDVI_5', 'NDVI_6', 'NDVI_7', 'NDVI_stand_dev',
       'seasonal_crops'],
      dtype='object')

In [153]:
csv = data1.drop(columns=['row_loc', 'col_loc','tile', 'NDVI_0', 'NDVI_1', 'NDVI_2',
       'NDVI_3', 'NDVI_4', 'NDVI_5','NDVI_6', 'NDVI_7', 'NDVI_stand_dev'])

In [154]:
csv.head()

,fid,seasonal_crops
0,14318083,0
1,14323456,0
2,7041501,1
3,6055968,0
4,11943279,0


In [155]:
csv_predicted_class = csv.drop(columns=['fid'])

In [156]:
(unique, counts) = np.unique(csv_predicted_class, return_counts=True)

In [157]:
frequencies = np.asarray((unique, counts)).T
print(frequencies)

[[       0  1605187]
 [       1 12720683]]


In [158]:
from osgeo import gdal

In [159]:
step1 = gdal.Open('data/00/field_id.tif', gdal.GA_ReadOnly)

In [160]:
from PIL import Image
Image.MAX_IMAGE_PIXELS = 14325870
im = Image.open('data/00/field_id.tif')
im = np.array(im)
im.shape

(4251, 3370)

In [161]:
im

array([[       0,        1,        2, ...,     3367,     3368,     3369],
       [    3370,     3371,     3372, ...,     6737,     6738,     6739],
       [    6740,     6741,     6742, ...,    10107,    10108,    10109],
       ...,
       [14315760, 14315761, 14315762, ..., 14319127, 14319128, 14319129],
       [14319130, 14319131, 14319132, ..., 14322497, 14322498, 14322499],
       [14322500, 14322501, 14322502, ..., 14325867, 14325868, 14325869]],
      dtype=int32)

In [162]:
GT_input = step1.GetGeoTransform()


In [163]:
print(GT_input)

(693559.1091246363, 10.0, 0.0, 133265.48983932866, 0.0, -10.0)


In [164]:
from affine import Affine

In [165]:
transf = (693559.1091246363, 10.0, 0.0, 133265.48983932866, 0.0, -10.0)
afn = Affine.from_gdal(*transf)

In [166]:
step2 = step1.GetRasterBand(1)

In [167]:
img_as_array = step2.ReadAsArray()

In [168]:
x =img_as_array

In [169]:
size1,size2=img_as_array.shape

In [170]:
import numpy as np
output=np.zeros(shape=(size1,size2))

In [171]:
output.shape

(4251, 3370)

In [172]:
print("x", str(x))

x [[       0        1        2 ...     3367     3368     3369]
 [    3370     3371     3372 ...     6737     6738     6739]
 [    6740     6741     6742 ...    10107    10108    10109]
 ...
 [14315760 14315761 14315762 ... 14319127 14319128 14319129]
 [14319130 14319131 14319132 ... 14322497 14322498 14322499]
 [14322500 14322501 14322502 ... 14325867 14325868 14325869]]


In [173]:
x_flatten = x.flatten()

In [174]:
df_x = pd.DataFrame(x_flatten, columns = ['fid'])

In [175]:
df_x.shape

(14325870, 1)

In [176]:
df_x.head()

,fid
0,0
1,1
2,2
3,3
4,4


In [177]:
csv_0_merged = pd.merge(df_x, csv, on=["fid"], how='left')

In [178]:
csv_0_merged.head()

,fid,seasonal_crops
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0


In [179]:
csv_0_merged.shape

(14325870, 2)

In [180]:
csv_0_merged_reduced = csv_0_merged.drop(columns=['fid'])

In [181]:
csv_0_merged_reduced.head()

,seasonal_crops
0,0
1,0
2,0
3,0
4,0


In [182]:
csv_0_merged_reduced.shape

(14325870, 1)

In [183]:
(unique, counts) = np.unique(csv_0_merged_reduced, return_counts=True)

In [184]:
frequencies_2 = np.asarray((unique, counts)).T
print(frequencies_2)

[[       0  1605187]
 [       1 12720683]]


In [185]:
output = csv_0_merged_reduced.to_numpy()

In [186]:
array_output = np.reshape(output, (4251, 3370))

In [187]:
import rasterio

In [188]:
dst_crs="EPSG:32636"

In [189]:
array_output.shape[1]

3370

In [190]:
with rasterio.open(
    'NDVI_Standard_dev_Farm_Africa_data_EPSG_32636_2019_seasonal_crops.tif',
    'w',
    driver='GTiff',
    height=array_output.shape[0],
    width=array_output.shape[1],
    count=1,
    dtype=np.float32,
    crs=dst_crs,
    transform=afn,
) as dest_file:
    dest_file.write(array_output, 1)
dest_file.close()